In [1]:
import tkinter as tk
import tkinter.ttk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np
import cv2
import skimage as ski
from skimage.color import rgb2gray
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import pandas as pd
import tables as tb

In [2]:
no_slices = 155

glioma_masks =  tb.open_file(r"C:\Users\user\Documents\UWA\Honours in Computer Science and Software Engineering\CITS4402 - Computer Vision\Project\archive\BraTS2020_training_data\content\data\volume_1_slice_0.h5", mode="r").root.mask.read()
glioma_images =  tb.open_file(r"C:\Users\user\Documents\UWA\Honours in Computer Science and Software Engineering\CITS4402 - Computer Vision\Project\archive\BraTS2020_training_data\content\data\volume_1_slice_0.h5", mode="r").root.image.read()

print(glioma_images.shape)
glioma_slices_masks = glioma_masks[np.newaxis,:]
glioma_slices_images = glioma_images[np.newaxis,:]

for slice in range(1,no_slices):
    
    next_glioma_mask =  tb.open_file(r"C:\Users\user\Documents\UWA\Honours in Computer Science and Software Engineering\CITS4402 - Computer Vision\Project\archive\BraTS2020_training_data\content\data\volume_1_slice_"+str(slice)+".h5", mode="r").root.mask.read()
    next_glioma_image =  tb.open_file(r"C:\Users\user\Documents\UWA\Honours in Computer Science and Software Engineering\CITS4402 - Computer Vision\Project\archive\BraTS2020_training_data\content\data\volume_1_slice_"+str(slice)+".h5", mode="r").root.image.read()
    
    glioma_slices_masks = np.vstack((glioma_slices_masks,next_glioma_mask[np.newaxis,:]))
    glioma_slices_images = np.vstack((glioma_slices_images,next_glioma_image[np.newaxis,:]))

(240, 240, 4)


In [3]:
# Function to create a merged mask from the list of masks uploaded from the dataset
# returns the merged mask
def merge_masks2(mask_array):

    # merge non-overlapping masks by addition
    merge_one = mask_array[:,:,0] + mask_array[:,:,1] + mask_array[:,:,2]
   
    return merge_one


In [20]:
# Function to determine the maximum tumour area present in a slice of an idividual MRI Volume 
def Maximum_tumout_area(masks):
    # set the number  of slices in a volume, set an empty list to accumulate all the glioma volume in voxels
    no_slices=masks.shape[0]
    slices_count= []

    # Loop over all slices calulating the glioma volume in each slice
    for slice in range(no_slices):
        #set the slice count to zero
        slice_count = 0

        # Obtain the image of the three masks collectively containing the whole tumour in one image 
        merged_mask = merge_masks2(glioma_slices_masks[slice,:,:,:])
        voxel_count = np.count_nonzero(merged_mask > 0) #calculate the number of  non zero voxels in the image of the glioma

        #Append the voxel count to a list
        slices_count.append(voxel_count)

total_tumor_count = Maximum_tumout_area(glioma_slices_masks)
max_vol = max(slices_count)
print(f"Maximum Volume (Voxels) in one slice is : {max_vol}")

            

Maximum Volume (Voxels) in one slice is : 5048


In [15]:
# Determine the maximum diameter of the tumour in a volume using Principle Component Analysis (PCA)
# take as a parameter all 3 masks dowloaded from Kaggle and returns a list of the diqmeters of the tumor for each slice.
def tumour_diameter(mask):
    # create a PCA object (no of components is one which returns the variance for only one eigenvector), 
    # and list to store the tumor diameter for a slice
    glioma_pca = PCA(n_components=1)
    slices_diameter = []
    
    # loop to determine the standard deviation, from PCA, as an approximation to the radius for eaxch slice of a volume
    for slice in range(no_slices):
    
        # create list to transform x and y co-ordinates in the mask image (mask image is binarized) 
        # to a list of x and y co-ordinates of pixels in the image having a value of 1.
        mask_newaxis = []
    
        # call merge_mask2 to return an image of the overlayed masks
        merged_mask = merge_masks2(mask[slice,:,:,:])
    
        # Transform the x and y coordinates in the image into a list of lists where the nested list contains the x any y 
        # co-ordinates in the image of all pixels whose value is 1.
        for x in range(merged_mask.shape[0]):
            for y in range(merged_mask.shape[1]):
                if merged_mask[x,y] == 1:
                    mask_newaxis.append([x,y])
                elif merged_mask[x,y] == 0:
                    continue
    
        # if no tumour is detected in the mask (ie no value of 1) then append 0 to the list of radius length for that slice
        if len(mask_newaxis) == 0:
            slices_diameter.append(0)
            continue
    
        # fit the tranformed x and y co-ordinates to the PCA object and get the variance
        glioma_pca.fit(mask_newaxis)
        variance = glioma_pca.explained_variance_
    
        # Append the diameter to the diameters list as twice the radius
        slices_diameter.append((variance[0]**0.5)*2)

    return slices_diameter
    
diameters = tumour_diameter(glioma_slices_masks)
max_tumor_diameter = max(diameters)
print(f"The maximum Diameter of a Glioma in this volume is {round(max_tumor_diameter,2)} mm")

The maximum Diameter of a Glioma is this volume is 48.92 mm


In [19]:
# Function to transform the list of x and y co-ordinates returned by the openCV findContours method
# into their corresponding pixels on an image, ie create a new image wioth the contour displayed in 2D
def transform_contour(contours):
    # Set the width and height of the image to be created, and create a blank image of zeroes
    image_w=240
    image_h=240
    contour_image = np.zeros((image_w,image_h))

    # Loop over all the x and y co-ordinates in the co-ordinate list, Set the corresponding x and y
    # co-ordinates to 255, ie produce a white contour on a black background (binary image)
    for contour in range(len(contours[0])):
        contour_image[contours[0][contour][0][1],contours[0][contour][0][0]]=255
    
    # Return the image of the contour
    return contour_image

# Function to produce a contour 5 pixels in width, of the brain (cerebral cortex), by producing a contour of the brain
# within a slice; storing the image of the contour; then subtracting this contour from the original image and producing
# another contour from the reduced image; then adding this image to the original contour. This process is repeated unitl 
# the image of the stored contours consists of 5 contours, hence being 5 pixels wide,
def merge_contours(image):
    #set variables for the width and height of the image, and the number or layers required for the combined contour
    image_w=240
    image_h=240
    outer_layers = 5

    # Convert the greyscale image of the brain into a binarized format, with all values above zero
    # set to 255 ie the brain matter will turn completeley white creating a sharp contrast at the 
    # border.
    ret, image_th = cv2.threshold(image,0,255, cv2.THRESH_BINARY)

    # convert the image to the correct datatype and create a blank image to store the created contours
    image_uint8 = image_th.astype(np.uint8)
    collected_contours = np.zeros((image_w,image_h))

    
    # Loop to calculate the contours of the brain image (outer contour); accumlate the contours in the
    # one image; subtract the produced contour form the image and repeat the procdedure using the reduced image
    # until 5 layers of contours have been added.
    for cont_num in range(outer_layers):
        # method to return the contours from the openCV findContours method. The parameter RETR_EXTERNAL ensure
        # only co-ordinates of the pixels in the outer contour of the brain image are returned. If no contour is found
        # there is no tumor in the slice the the function continues to process the next slice.
        contours, _ = cv2.findContours(image_uint8,cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)
        if len(contours) == 0: continue

        # Call the transform_contours function to return an image of the contour.
        contour_image = transform_contour(contours)

        # Add the image of the contour to the blank image and accumulate the remaining 4 cnotours on subsequent loop iterations
        collected_contours += contour_image

        # subtract the contour from the image of the brain to produce a new image reduced in size by one pixel around the edge/perimeter
        # and change the datatype to uint8 to promote continued processing of the image
        reduced_image = image_uint8 - contour_image
        image_uint8 = reduced_image.astype(np.uint8)

    # return the combined image of the contours
    return collected_contours



# function to determine the maximum volume of the Glioma within a MRI volume, by using a combined mask image of the Glioma
# as a blocking/filtering mask on an image of the cerrebral cortex. 
def glioma_cortex_invasion(volume, masks):
    # Set the variable to accumulate the numebr of voxels invaded by the glioma in the cerebral cortex across the MRI volume 
    total_voxels = 0

    # loop to process each slice in an MRI volume calculating the glioma occupied part ot the cerebral cortex for each slice
    for slice in range(volume.shape[0]):
        T1_native = merge_contours(volume[slice,:,:,1]) # get the 5 pixel thick contour of the brain from a slice
        glioma_merged_mask = merge_masks2(masks[slice,:,:,:]) # Get the image of the merged masks
        glioma_overlap = cv2.bitwise_and(T1_native.astype(np.uint8), glioma_merged_mask) # Return the overlap of the glioma and the cerebral cortex
        voxel_count = np.count_nonzero(glioma_overlap == 1) # calculate the nuber of voxels labelled 1 (the overlap)
        total_voxels += voxel_count
    
    return total_voxels
    
glioma_cotex_invasion = glioma_cortex_invasion(glioma_slices_images,glioma_slices_masks)
print(f"The total number of voxels occupied by the Glioma in the Cerebral Cortex is {glioma_cotex_invasion}")

The total number of voxels occupied by the Glioma in the Cerebral Cortex is 5635
